# [실습] LangChain Expression Language


LangChain Expression Language(LCEL)는 랭체인에서 체인을 간결하게 구성하는 문법입니다.   
먼저, LCEL에서 체인이 구성되는 기본적인 구조에 대해 알아봅시다.


In [1]:
!pip install openai langchain langchain_openai -q

In [1]:
import os
from dotenv import load_dotenv
from pathlib import Path

env_path = Path("/Users/blueno/UNO/SKALA/SKALA/.env")
load_dotenv(dotenv_path=env_path, override=True)

# 환경 변수 확인
openai_api_key = os.getenv("OPENAI_API_KEY")

기존에는 `LLM.invoke(prompt.format(___))` 형태로 실행을 했었는데요.    
LCEL의 가장 큰 특징은, Chain의 구성 요소를 **|**  (파이프)로 연결하여 한 번에 실행한다는 점입니다. 예시를 보겠습니다.

In [5]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


# topic에 대한 영어 농담을 하고, 이것이 왜 농담인지 한국어로 설명하세요.
fun_chat_template = ChatPromptTemplate.from_messages([
    ('user',"""tell me an English joke about {topic},
also, explain in Korean why it is fun for english-speakers.
Include translation of the joke.""")
])


llm=ChatOpenAI(temperature=0.5, model='gpt-4o-mini',max_tokens=1000)
# LLM 유머는 지능에 비례



In [6]:
response = llm.invoke(fun_chat_template.format_messages(topic='AI'))
response

AIMessage(content="**Joke:**\nWhy did the robot go on a diet?  \nBecause he had too many bytes!\n\n**Translation:**\n로봇이 왜 다이어트를 시작했나요?  \n너무 많은 바이트를 먹었기 때문이에요!\n\n**Explanation in Korean:**\n이 농담은 영어 사용자에게 재미있는 이유는 'bytes'라는 단어의 이중 의미 때문입니다. 'Bytes'는 컴퓨터 데이터의 단위로, 로봇과 관련이 깊습니다. 동시에 'bite'와 발음이 비슷하여, 음식의 한 입을 의미하기도 합니다. 따라서 로봇이 다이어트를 하는 이유가 '너무 많은 바이트를 먹었다'는 말은 컴퓨터 데이터와 음식의 양을 동시에 언급하여 유머를 만들어냅니다. 이러한 말장난은 영어에서 흔히 사용되는 유머 스타일 중 하나입니다.", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 187, 'prompt_tokens': 35, 'total_tokens': 222, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-48bb9fd6-2d5b-4f3f-9667-1c88a8280696-0', usage_metadata={'input_tokens': 35, 'outp

In [7]:
print(response.content)

**Joke:**
Why did the robot go on a diet?  
Because he had too many bytes!

**Translation:**
로봇이 왜 다이어트를 시작했나요?  
너무 많은 바이트를 먹었기 때문이에요!

**Explanation in Korean:**
이 농담은 영어 사용자에게 재미있는 이유는 'bytes'라는 단어의 이중 의미 때문입니다. 'Bytes'는 컴퓨터 데이터의 단위로, 로봇과 관련이 깊습니다. 동시에 'bite'와 발음이 비슷하여, 음식의 한 입을 의미하기도 합니다. 따라서 로봇이 다이어트를 하는 이유가 '너무 많은 바이트를 먹었다'는 말은 컴퓨터 데이터와 음식의 양을 동시에 언급하여 유머를 만들어냅니다. 이러한 말장난은 영어에서 흔히 사용되는 유머 스타일 중 하나입니다.


-----------
LCEL의 인터페이스는 조금 다릅니다.

템플릿과 llm 모델을 설정하고, 이를 하나로 묶어 체인을 생성합니다.

In [8]:
joke = fun_chat_template | llm
# shift + \ = |||||
joke

ChatPromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], input_types={}, partial_variables={}, template='tell me an English joke about {topic},\nalso, explain in Korean why it is fun for english-speakers.\nInclude translation of the joke.'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x10ea98890>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x10eabdd50>, root_client=<openai.OpenAI object at 0x10eaa12d0>, root_async_client=<openai.AsyncOpenAI object at 0x10ea982d0>, model_name='gpt-4o-mini', temperature=0.5, model_kwargs={}, openai_api_key=SecretStr('**********'), max_tokens=1000)

이후, 체인의 invoke를 실행하며 입력 포맷을 전달하면, 순서대로 체인이 실행되며 최종 결과로 연결됩니다.    
입력 포맷은 Dict 형식으로 전달합니다.

In [9]:
response = joke.invoke({'topic':'apple'})
# invoke에 대한 입력은 딕셔너리로 처리 ':'
# 단, 매개 변수가 1개일 때는
# joke.invoke('apple')과 같이 문자열로 입력할 수도 있음
response

AIMessage(content='**Joke:**\nWhy did the apple stop in the middle of the road?  \nBecause it ran out of juice!\n\n**Translation in Korean:**\n왜 사과가 도중에 멈췄을까요?  \n즉석에서 주스가 다 떨어졌기 때문이에요!\n\n**Explanation in Korean:**\n이 농담은 영어에서 "juice"라는 단어의 이중 의미 때문에 재미있습니다. "Juice"는 사과 주스를 의미할 뿐만 아니라, 자동차의 연료나 에너지를 뜻하는 비유적 표현으로도 사용됩니다. 따라서 사과가 "주스"가 다 떨어져서 멈췄다는 것은 실제로 사과가 주스가 없어서 멈춘 것처럼 들리지만, 자동차가 연료가 떨어져서 멈춘 것과 같은 의미로 해석될 수 있습니다. 이처럼 언어의 중의성을 이용한 유머가 영어 화자들에게 재미를 줍니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 208, 'prompt_tokens': 35, 'total_tokens': 243, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-b9fd545b-5de8-4441-add4-6f6203574bb8-0', usage_metadata={'inpu

In [10]:
print(response.content)

**Joke:**
Why did the apple stop in the middle of the road?  
Because it ran out of juice!

**Translation in Korean:**
왜 사과가 도중에 멈췄을까요?  
즉석에서 주스가 다 떨어졌기 때문이에요!

**Explanation in Korean:**
이 농담은 영어에서 "juice"라는 단어의 이중 의미 때문에 재미있습니다. "Juice"는 사과 주스를 의미할 뿐만 아니라, 자동차의 연료나 에너지를 뜻하는 비유적 표현으로도 사용됩니다. 따라서 사과가 "주스"가 다 떨어져서 멈췄다는 것은 실제로 사과가 주스가 없어서 멈춘 것처럼 들리지만, 자동차가 연료가 떨어져서 멈춘 것과 같은 의미로 해석될 수 있습니다. 이처럼 언어의 중의성을 이용한 유머가 영어 화자들에게 재미를 줍니다.


## 실습) 매개변수가 2개인 Prompt-LLM Chain 생성하기   
임의의 ChatPromptTemplate를 만들고, 2개의 매개변수를 받도록 구성하여 체인을 만들고 실행하세요.

In [11]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate


# topic에 대한 영어 농담을 하고, 이것이 왜 농담인지 한국어로 설명하세요.
fun_chat_template1 = ChatPromptTemplate.from_messages([
    ('user',"""tell me an English joke about {topic},
also, explain in Korean why it is fun for english-speakers.
Include translation of the joke.""")
])

fun_chat_template2 = ChatPromptTemplate.from_messages([
    ('user',"""tell me an Korean joke about {topic},
also, explain in Korean why it is fun for korean-speakers.
Include translation of the joke.""")
])


llm=ChatOpenAI(temperature=0.5, model='gpt-4o',max_tokens=1000)
# LLM 유머는 지능에 비례

In [12]:
response1 = llm.invoke(fun_chat_template1.format_messages(topic='AI'))
response2 = llm.invoke(fun_chat_template2.format_messages(topic='AI'))

print(response1.content)
print(response2.content)

**Joke:**

Why did the AI go to art school?

Because it wanted to learn how to draw its own conclusions!

**Translation of the Joke:**

왜 AI가 미술 학교에 갔을까요?

자신의 결론을 그리는 법을 배우고 싶었기 때문이에요!

**Explanation in Korean:**

이 농담이 재미있는 이유는 "draw"라는 단어의 중의적 의미 때문입니다. 영어에서 "draw"는 "그리다"라는 의미도 있지만, "결론을 내리다"라는 의미로도 사용됩니다. AI가 미술 학교에 간다는 상황이 웃긴 이유는 AI가 실제로 그림을 그리기 위해 미술 학교에 간 것이 아니라, "결론을 내리는 법"을 배우기 위해 갔다는 뜻으로 해석되기 때문입니다. 이러한 언어유희가 영어 화자들에게 유머를 줍니다.
Here's a Korean joke about AI:

---

**Joke:**

왜 인공지능(AI)이 한국어를 배우기 어려워할까요?

왜냐하면, "눈"이 두 가지 의미가 있어서요. AI가 "눈이 온다"라고 하면, 진짜 눈이 오는 건지, 아니면 눈이 걸어오는 건지 헷갈려 하거든요!

---

**Translation:**

Why is it difficult for AI to learn Korean?

Because the word "눈" (nun) has two meanings. When AI hears "눈이 온다" (nun-i onda), it gets confused whether it's snowing or eyes are coming!

---

**Explanation in Korean:**

이 농담은 한국어의 동음이의어를 이용한 것입니다. "눈"이라는 단어는 한국어에서 'snow'와 'eye' 두 가지 의미를 가지고 있습니다. 그래서 "눈이 온다"라는 표현은 문맥에 따라 '눈(snow)이 온다'와 '눈(eye)이 온다'로 해석될 수 있습니다. 인공지능이 이러한 한국어

In [138]:
joke = fun_chat_template1 | fun_chat_template1 | llm
# shift + \ = |||||
print(joke)


first=ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me an English joke about {topic},\nalso, explain in Korean why it is fun for english-speakers.\nInclude translation of the joke.'))]) middle=[ChatPromptTemplate(input_variables=['topic'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['topic'], template='tell me an English joke about {topic},\nalso, explain in Korean why it is fun for english-speakers.\nInclude translation of the joke.'))])] last=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x2a37c0dd0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x2a38c8410>, root_client=<openai.OpenAI object at 0x2a382e290>, root_async_client=<openai.AsyncOpenAI object at 0x2a37c0e50>, model_name='gpt-4o', temperature=0.5, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=1000)


LCEL의 체인에는 **파서(Parser)** 를 추가할 수 있습니다.    
파서는 출력 형식을 변환합니다.

StrOutputParser : 출력 결과를 String 형식으로 변환합니다.

In [139]:
from langchain_core.output_parsers import StrOutputParser

recipe_template=ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 조리법을 아는 쉐프입니다.'),
    ('user','저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!')
])

In [140]:
recipe_chain = recipe_template | llm | StrOutputParser()
response = recipe_chain.invoke({'ingredient':'콜라'})
print(response)

콜라를 사용한 외국 음식 중 하나로는 "콜라 치킨"을 추천드립니다. 이 요리는 특히 중국과 미국 남부에서 인기를 끌며, 콜라의 단맛과 간장의 짭짤함이 어우러져 독특한 맛을 냅니다. 다음은 콜라 치킨을 만드는 간단한 방법입니다.

### 재료:
- 닭고기 (닭 날개나 닭 다리) 500g
- 콜라 1컵 (약 240ml)
- 간장 1/4컵 (약 60ml)
- 다진 마늘 2쪽
- 생강 1작은술 (선택 사항)
- 식용유 1큰술
- 소금과 후추 약간
- 참깨와 다진 파 (장식용)

### 조리 방법:
1. **닭고기 준비**: 닭고기를 깨끗이 씻고, 소금과 후추로 간을 합니다.

2. **기름 두르기**: 큰 프라이팬이나 냄비에 식용유를 두르고 중간 불로 가열합니다.

3. **닭고기 익히기**: 닭고기를 팬에 넣고 겉면이 노릇해질 때까지 구워줍니다. 이 과정은 약 5~7분 정도 소요됩니다.

4. **양념 추가**: 닭고기가 노릇해지면 다진 마늘과 생강을 넣고 1분 정도 더 볶습니다.

5. **콜라와 간장 추가**: 콜라와 간장을 팬에 부어줍니다. 모든 재료가 잘 섞이도록 저어줍니다.

6. **조리**: 팬의 뚜껑을 덮고 약한 불에서 20~30분 동안 졸입니다. 소스가 걸쭉해지고 닭고기에 잘 배어들 때까지 조리합니다.

7. **마무리**: 불을 끈 후 참깨와 다진 파를 뿌려 장식합니다.

이제 콜라 치킨이 완성되었습니다! 밥이나 면과 함께 곁들여 드시면 좋습니다. 즐거운 요리 시간 되세요!


파서는 스트링이 아닌 json 형식으로도 만들 수 있습니다.   
프롬프트에서 형식을 요청하고, 이를 파서와 결합하여 변환하는 방식입니다.

In [141]:
from langchain_core.output_parsers import JsonOutputParser

jsonparser = JsonOutputParser()

In [142]:
jsonparser

JsonOutputParser()

In [143]:
jsonparser.get_format_instructions()

'Return a JSON object.'

In [144]:
recipe_template=ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 {ingredient}를 이용한 환상적인 파인 다이닝을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요.''')
])

recipe_chain = recipe_template | llm | jsonparser


In [145]:
response = recipe_chain.invoke({'ingredient':'콜라'})
response

{'recipe': {'name': '콜라 글레이즈드 덕 브레스트',
  'cuisine': '퓨전',
  'servings': 2,
  'ingredients': [{'item': '오리 가슴살', 'quantity': '2 pieces'},
   {'item': '콜라', 'quantity': '1 cup'},
   {'item': '발사믹 식초', 'quantity': '2 tablespoons'},
   {'item': '꿀', 'quantity': '1 tablespoon'},
   {'item': '간장', 'quantity': '1 tablespoon'},
   {'item': '마늘', 'quantity': '2 cloves, minced'},
   {'item': '생강', 'quantity': '1 teaspoon, grated'},
   {'item': '소금', 'quantity': 'to taste'},
   {'item': '후추', 'quantity': 'to taste'},
   {'item': '올리브 오일', 'quantity': '2 tablespoons'}],
  'instructions': ['오리 가슴살에 소금과 후추로 간을 합니다.',
   '큰 팬에 올리브 오일을 두르고 중간 불로 가열합니다.',
   '오리 가슴살을 껍질이 아래로 가게 놓고 약 6-8분간 바삭하게 구워줍니다.',
   '뒤집어서 반대쪽도 4-5분 정도 구워줍니다. 오리 가슴살을 팬에서 꺼내어 따로 둡니다.',
   '같은 팬에 마늘과 생강을 넣고 약 1분간 볶아 향을 냅니다.',
   '콜라, 발사믹 식초, 꿀, 간장을 넣고 잘 섞어줍니다.',
   '소스를 중불에서 졸여 글레이즈가 되도록 합니다. 약 10-15분 소요됩니다.',
   '오리 가슴살을 팬에 다시 넣고 글레이즈로 코팅되도록 양쪽 면을 잘 뒤집어줍니다.',
   '완성된 오리 가슴살을 얇게 슬라이스하여 접시에 담고 남은 글레이즈를 위에 뿌려줍니다.'],
  'presentation': 

In [60]:
response['dish_name']

'콜라 글레이즈 오리 가슴살'

Json으로 파싱하는 방법은 활용도가 높지만, 실행할 때마다 결과뿐만 아니라 형식도 달라진다는 문제가 있습니다.

In [ ]:
response = recipe_chain.invoke({'ingredient':'콜라'})
response

## Pydantic을 이용해 확실한 형식 지정하기

pydantic은 데이터 형식에 제약조건을 두고 이를 준수하는지 검증하는 라이브러리입니다.


In [129]:
from pydantic import BaseModel, Field
# pydantic 연동

class Recipe(BaseModel):
    name: str = Field(description="음식 이름")
    # name: 문자열, 설명은 "음식 이름"
    difficulty: str = Field(description="만들기의 난이도")
    kick: str = Field(description='맛의 포인트')

    origin: str = Field(description="원산지")
    ingredients: list[str] = Field(description="재료")
    # ingredients: 문자열 리스트, 설명은 "재료"

    instructions: list[str] = Field(description="조리법")
    tip: str = Field(description='조리 과정 팁')


In [130]:
parser = JsonOutputParser(pydantic_object=Recipe)

In [131]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "\uc74c\uc2dd \uc774\ub984", "title": "Name", "type": "string"}, "difficulty": {"description": "\ub9cc\ub4e4\uae30\uc758 \ub09c\uc774\ub3c4", "title": "Difficulty", "type": "string"}, "kick": {"description": "\ub9db\uc758 \ud3ec\uc778\ud2b8", "title": "Kick", "type": "string"}, "origin": {"description": "\uc6d0\uc0b0\uc9c0", "title": "Origin", "type": "string"}, "ingredients": {"description": "\uc7ac\ub8cc", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "instructions": {"descript

In [ ]:
# depreciated
# refined_json_instructions = parser.get_format_instructions().encode().decode('unicode_escape')
# print(refined_json_instructions)

In [71]:
recipe_template2 =ChatPromptTemplate.from_messages([
    ('system','당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.'),
    ('user','''저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!
    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.
     {instruction}''')
])

recipe_chain2 = recipe_template2 | llm | parser


In [72]:
recipe_template2

ChatPromptTemplate(input_variables=['ingredient', 'instruction'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='당신은 전세계의 이색적인 퓨전 조리법의 전문가입니다.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ingredient', 'instruction'], template='저는 {ingredient}를 이용한 환상적인 외국 음식을 만들고 싶습니다. 추천해주세요!\n    레시피에 대한 정보를 JSON 형식으로 출력해주세요. 결과는 한국어로 작성하세요.\n     {instruction}'))])

In [73]:
recipe_chain2.invoke({'ingredient':'생강', 'instruction':parser.get_format_instructions}) #refined_json_instructions})

{'요리명': '생강 테리야키 연어',
 '재료': [{'이름': '연어 필레', '양': '4조각'},
  {'이름': '생강', '양': '2큰술', '준비방법': '다진 것'},
  {'이름': '간장', '양': '1/4컵'},
  {'이름': '미림', '양': '1/4컵'},
  {'이름': '설탕', '양': '2큰술'},
  {'이름': '마늘', '양': '2쪽', '준비방법': '다진 것'},
  {'이름': '올리브 오일', '양': '1큰술'},
  {'이름': '파', '양': '1대', '준비방법': '얇게 썬 것'},
  {'이름': '참깨', '양': '1큰술'}],
 '조리법': ['작은 볼에 다진 생강, 간장, 미림, 설탕, 다진 마늘을 넣고 잘 섞어 테리야키 소스를 만드세요.',
  '연어 필레를 준비한 테리야키 소스에 최소 30분간 재워 두세요.',
  '팬에 올리브 오일을 두르고 중간 불로 가열하세요.',
  '재운 연어를 팬에 올리고 양면을 각각 3-4분씩 구워 속까지 익히세요.',
  '연어를 접시에 담고 남은 소스를 팬에 넣어 살짝 끓인 후 연어 위에 뿌리세요.',
  '썬 파와 참깨를 뿌려 장식하세요.'],
 '조리시간': '총 45분',
 '난이도': '중간'}

# Structured Output
LangChain의 Structured_Output 기능을 사용할 수도 있습니다.

In [127]:
structured_llm = llm.with_structured_output(Recipe)
structured_llm.invoke("생강으로 만들 수 있는 요리 레시피 알려주세요.")

Recipe(name='생강차', difficulty='쉬움', kick='생강의 매운맛과 꿀의 달콤함', origin='한국', ingredients=['생강 50g', '물 500ml', '꿀 2큰술', '레몬 슬라이스 (선택사항)'], instructions=['생강을 깨끗이 씻어 얇게 슬라이스합니다.', '냄비에 물과 생강을 넣고 끓입니다.', '물이 끓기 시작하면 불을 줄이고 10분간 더 끓입니다.', '불을 끄고 생강 조각을 걸러냅니다.', '컵에 생강차를 붓고 꿀을 섞습니다.', '레몬 슬라이스를 추가하여 장식합니다.'], tip='생강을 얇게 슬라이스할수록 더 많은 맛을 우려낼 수 있습니다. 꿀은 기호에 따라 양을 조절하세요.')

In [153]:
structured_llm = llm.with_structured_output(Recipe, method='json_mode')
structured_llm.invoke("생강으로 만들 수 있는 요리 레시피 알려주세요."+parser.get_format_instructions())

Recipe(name='생강차', difficulty='쉬움', kick='따뜻하고 매콤한', origin='한국', ingredients=['생강 50g', '물 500ml', '꿀 2큰술', '레몬 슬라이스 (선택사항)'], instructions=['생강을 껍질을 벗기고 얇게 썬다.', '냄비에 물과 생강을 넣고 끓인다.', '물이 끓기 시작하면 불을 줄이고 10분간 더 끓인다.', '생강을 체로 걸러내고, 꿀을 넣어 잘 섞는다.', '컵에 생강차를 따르고, 레몬 슬라이스를 추가하여 제공한다.'], tip='생강차는 감기 예방에 좋으며, 레몬을 추가하면 비타민 C를 보충할 수 있습니다.')

이제, 댓글이 주어지면 댓글의 답변을 생성해 보겠습니다.

In [83]:
import pandas as pd
reviews = pd.read_csv('./reviews.csv')
print(reviews.shape)
reviews.head()

(50, 3)


/opt/homebrew/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


,Num,Review,Label
0,1,그닥 맛있고 좋은 고기인지는 모르겠내요 테이블 나누어서 두팀 받는데 옆 테이블 시끄...,-1
1,2,적당히 좋은 소고기를 싸지 않은 가격에 맛볼 수 있다. 하지만 인기나 리뷰에 비해선...,-1
2,3,기름진맛 역시맛있네요 많이먹긴 힘들지만 맛있는 소고기집인건 틀림없어요,1
3,4,"지인이 추천하고 짬뽕맛집이래서 찾아갔는데, 주차도 골목길에 해야되고 맛도 별로네요",-1
4,5,매운단계별짬뽕과 불향가득짜장 직접만든소스가별미인탕수육 제입맛에는딱맞아서자주찾게되요^^*,1


In [85]:
reply_template = ChatPromptTemplate.from_messages([
    ('system','''당신은 레스토랑의 주인입니다.
고객이 다음과 같은 리뷰를 남겼을 때, 답변을 작성해 주세요.
첫 문장은 가상의 레스토랑 이름과 함께 인사하는 내용을 포함하세요.
고객의 의견에 매우 공감하여 답변하고, 부정적인 피드백은 사과하세요.
새로운 메뉴나 프로모션을 홍보호가, 재방문을 기원하세요.
밝고 유쾌한 톤으로 작성하고, 이모지를 매우 많이 추가하세요.

'''),
    ('user','''{review}''')

])

In [89]:
reply_chain = reply_template | llm | StrOutputParser()
reply_chain.invoke({'review':reviews['Review'][0]})

'안녕하세요! 😊 "고기천국"에 방문해 주셔서 감사합니다!\n\n먼저, 저희 고기의 맛과 품질이 기대에 미치지 못한 점 정말 죄송합니다. 😔 고객님께 최고의 맛을 선사하기 위해 더욱 노력하겠습니다. 또한, 옆 테이블의 소음 때문에 불편을 겪으신 점도 사과드립니다. 다음 방문 시에는 더 조용하고 편안한 식사를 하실 수 있도록 신경 쓰겠습니다. 🙏\n\n혹시 새로운 메뉴로 나온 "특제 양념 소고기"를 시도해 보셨나요? 🍖 지금 첫 주문 시 10% 할인 프로모션도 진행 중이니 꼭 한번 맛보세요! 😊\n\n다시 뵙기를 진심으로 기대하며, 다음 방문 때는 더 나은 경험을 제공해 드리겠습니다. 행복한 하루 보내세요! 🌟✨'

입력이 여러 개인 경우, 이를 리스트로 만들고 batch()를 이용해 한꺼번에 입력할 수 있습니다.

In [87]:
reviews.loc[:5,'Review'].to_list()

['그닥 맛있고 좋은 고기인지는 모르겠내요 테이블 나누어서 두팀 받는데 옆 테이블 시끄러워서 짜증이 많이 나내요',
 '적당히 좋은 소고기를 싸지 않은 가격에 맛볼 수 있다. 하지만 인기나 리뷰에 비해선 평범한 수준. 이런 맛과 가성비의 소고기집은 동네마다 아주아주 많다.',
 '기름진맛 역시맛있네요 많이먹긴 힘들지만 맛있는 소고기집인건 틀림없어요',
 '지인이 추천하고 짬뽕맛집이래서 찾아갔는데, 주차도 골목길에 해야되고 맛도 별로네요',
 '매운단계별짬뽕과 불향가득짜장 직접만든소스가별미인탕수육 제입맛에는딱맞아서자주찾게되요^^*',
 '신맛나지 않는 달콤한 탕수육도 맛있다 친절한 사장님도 추가 당분간 맛없는 배달짬뽕 먹을 생각하니 슬프다 사장님 쾌차하세요']

In [88]:
reply_chain.batch(reviews.loc[:5,'Review'].to_list())

["안녕하세요! 🌟 '맛있는 한우집'에 오신 것을 환영합니다! 🥩\n\n먼저, 저희 레스토랑에서의 경험이 기대에 미치지 못해 정말 죄송합니다. 😢 고기의 맛과 품질이 고객님의 기대에 부응하지 못한 점, 그리고 옆 테이블의 소음으로 불편을 겪으신 점에 대해 사과드립니다. 🙏\n\n고객님의 소중한 피드백을 바탕으로 더 나은 서비스를 제공하기 위해 노력하겠습니다. 저희는 최근에 새로운 메뉴를 준비하고 있답니다! 🍖✨ 다음번 방문 시, 특별한 할인 혜택도 제공해드릴게요! 🎉\n\n다시 한번 저희를 찾아주시면 더욱 만족스러운 경험을 제공할 수 있도록 최선을 다하겠습니다. 💪 다음 방문을 손꼽아 기다리고 있을게요! 감사합니다! 😊❤️\n\n맛있는 하루 되세요! 🍽️",
 "안녕하세요, '고기천국'에 오신 것을 환영합니다! 😊🥩\n\n소중한 리뷰 남겨주셔서 정말 감사합니다. 고객님께서 말씀하신 부분에 깊이 공감하며, 저희가 더 나은 경험을 제공하지 못한 점 진심으로 사과드립니다. 😔🙏\n\n저희는 고객님께서 특별하게 느끼실 수 있도록 더 다양한 메뉴와 특별한 프로모션을 준비 중입니다! 예를 들어, 곧 출시될 '특선 소고기 플래터'와 함께 다양한 사이드 메뉴를 할인된 가격에 제공할 예정이니 꼭 한번 방문해 주세요! 🎉🍽️\n\n다시 한번 찾아주신다면, 더 나은 서비스와 맛으로 보답드리겠습니다. 다음 방문을 손꼽아 기다리고 있을게요! 감사합니다! 💖🥳\n\n행복한 하루 보내세요! 🌟",
 "안녕하세요! 🌟 '소고기 천국'에 오신 것을 환영합니다! 🥩✨\n\n저희 소고기의 맛을 즐기셨다니 정말 기쁩니다! 😄 고소한 맛이 매력적이지만, 가끔은 조금 무거울 수 있죠. 이 점에 대해 사과드리며, 고객님의 소중한 의견에 감사드립니다. 🙏\n\n다음 번 방문 때는 좀 더 산뜻한 맛을 원하신다면, 저희가 새롭게 준비한 상큼한 샐러드 메뉴 🥗나 가벼운 와인 한 잔 🍷을 추천드리고 싶어요! 현재 진행 중인 '가족과 함께' 할인 프로모션도 놓치지 마세요! 👨\u200d👩\u200d👧\u200d

프롬프트를 잘 구성하거나, Schema를 사용한다면 답글의 형식을 통일할 수 있습니다.

<br><br>
## Runnables

Runnables는 LCEL의 기본 단위로, 입력을 받아 출력을 생성하는 기본 단위입니다.    
llm, prompt, chain 등이 모두 Runnable 구조에 해당합니다.

이번에는, 데이터 흐름을 제어하는 특별한 Runnable인   
RunnablePassthrough와 RunnableParallel을 이용해 체인을 구성해 보겠습니다.


<br><br>
### RunnablePassthrough
RunnablePassthrough는 체인의 직전 출력을 그대로 가져옵니다.

In [90]:
from langchain.schema.runnable import RunnablePassthrough

prompt1 = ChatPromptTemplate.from_template(
    "{director}의 대표 작품은 무엇입니까?")
chain1 = (
    prompt1
    | llm
    | StrOutputParser()
    | {'answer': RunnablePassthrough()})

response = chain1.invoke("스티븐 스필버그")
print(response)

{'answer': "스티븐 스필버그는 여러 걸작을 감독한 유명한 영화 감독입니다. 그의 대표 작품 중 일부는 다음과 같습니다:\n\n1. **죠스 (Jaws, 1975)** - 스릴러 장르의 고전으로, 해변 마을을 위협하는 상어를 다룹니다.\n2. **인디아나 존스 시리즈** - 고고학자 인디아나 존스의 모험을 그린 액션 어드벤처 영화들입니다.\n3. **ET (E.T. the Extra-Terrestrial, 1982)** - 외계 생명체와 어린 소년의 우정을 그린 감동적인 이야기입니다.\n4. **쥬라기 공원 (Jurassic Park, 1993)** - 공룡을 복원한 테마파크에서 벌어지는 사건을 다룬 SF 영화입니다.\n5. **쉰들러 리스트 (Schindler's List, 1993)** - 홀로코스트 동안 유대인을 구한 사업가 오스카 쉰들러의 실화를 바탕으로 한 드라마입니다.\n6. **라이언 일병 구하기 (Saving Private Ryan, 1998)** - 제2차 세계 대전을 배경으로 한 전쟁 영화로, 사실적인 전투 장면으로 유명합니다.\n\n이 외에도 스필버그는 수많은 작품을 통해 다양한 장르에서 뛰어난 연출력을 보여주었습니다."}


<br><br>
### RunnableParallel

RunnableParallel은 서로 다른 체인을 병렬적으로 실행합니다.

In [93]:
from langchain_core.runnables import RunnableParallel

prompt1 = ChatPromptTemplate.from_template( # 괄호 뒤에서 줄바꿈하면 다음 줄도 같은 줄로 인식
    "색깔을 하나 알려주세요, 색깔만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "음식을 하나 알려주세요, 음식만 출력하세요.")

chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()

chain3 = RunnableParallel(color = chain1, food = chain2)

chain3.invoke({}) # chain3.invoke()는 에러


{'color': '파랑', 'food': '김치찌개'}

<br><br><br>이번에는 LLM의 결과를 다음 LLM으로 연결해 보겠습니다.

In [106]:
prompt1 = ChatPromptTemplate.from_template("잭슨빌은 어느 나라의 도시입니까? 나라이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{country}의 대표적인 인물 3명을 나열하세요. 인물의 이름만 출력하세요."
)

chain1 = prompt1 | llm | StrOutputParser()
chain2 =(
    {"country": chain1} | prompt2 | llm | StrOutputParser()
)
chain2.invoke({})

'조지 워싱턴  \n에이브러햄 링컨  \n마틴 루터 킹 주니어'

RunnableParallel을 사용하면 chain1의 결과와 chain2의 결과를 함께 얻을 수 있습니다.   
이 때, chain1의 실행 결과를 chain2에 전달하는 방식으로 실행됩니다.

In [107]:
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()  # chain2 형식이 바뀜

chain3 = RunnableParallel(country = chain1).assign(people = chain2)

chain3.invoke({})

{'country': '미국', 'people': '조지 워싱턴  \n에이브러햄 링컨  \n마틴 루터 킹 주니어'}

## 실습) 체인 연결하기

위 코드들을 참고하여, LLM을 3개 연결해서 3개 결과를 출력해 보세요.   
체인 3개로 만들 수도 있고, RunnableParallel을 사용할 수도 있습니다.   

예시)    
1. 영화->감독
2. 감독->가장 유명한 배우
3. 배우->상을 받은 영화

In [123]:
# 예시 프롬프트
prompt1 = ChatPromptTemplate.from_template("{movie}의 감독은 누구입니까? 감독 이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template("{director}와 작업한 가장 유명한 배우는 누구인가요? 배우 이름만 출력하세요.")
prompt3 = ChatPromptTemplate.from_template("{actor}는 무슨 영화로 상을 받았나요?")



In [125]:
chain1 = prompt1 | llm | StrOutputParser()
chain2 = prompt2 | llm | StrOutputParser()
chain3 = prompt3 | llm | StrOutputParser()

chain4 = RunnableParallel(director = chain1).assign(actor = chain2).assign(award = chain3)

chain4.invoke('죠스')


{'director': '스티븐 스필버그',
 'actor': '톰 행크스',
 'award': '톰 행크스는 여러 영화로 상을 받았습니다. 그는 특히 아카데미 시상식에서 두 번의 남우주연상을 수상했습니다. 첫 번째는 1993년 영화 "필라델피아"로, 두 번째는 1994년 영화 "포레스트 검프"로 수상했습니다. 이 외에도 그는 다양한 영화로 골든 글로브, 에미상 등 여러 상을 받았습니다.'}

<br><br>
chain2에서 새로운 매개변수가 추가되는 경우는 어떻게 해야 할까요?

Lambda 함수를 통해, 입력 dict로부터 값을 선택합니다.

In [146]:
prompt1 = ChatPromptTemplate.from_template("{city}는 어느 나라의 도시인가요? 나라 이름만 출력하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{country}의 유명한 인물은 누가 있나요? {num} 명의 이름을 나열하세요. 사람 이름만 ,로 구분하여 나열하세요."
)

chain1 = prompt1 | llm | StrOutputParser()

chain2 = (
    RunnableParallel(country = chain1, num = lambda x:x['num'])
    # lambda x:f(x) --> x를 입력 받으면 f(x)를 return, x는 직전에 입력된 값
    # invoke에서 주어지는 dict를 전처리하기
    | prompt2
    | llm
    | StrOutputParser()
)

print(chain2.invoke({"city": "잭슨빌", "num": "3"}))

조지 워싱턴, 마틴 루터 킹 주니어, 마릴린 먼로


<br><br>
체인을 분리하고 RunnableParallel을 이용하면 중간 과정을 모두 출력할 수 있습니다.

In [147]:
chain4 = (prompt2
    | llm
    | StrOutputParser())

chain3 = RunnableParallel(country = chain1, num = lambda x:x['num']).assign(res = chain4)

chain3.invoke({"city": "코펜하겐", "num": "3"})

{'country': '덴마크', 'num': '3', 'res': '한스 크리스티안 안데르센, 쇠렌 키르케고르, 마드스 미켈센'}

<br><br><br>JsonOutputParser를 쓴다면 아래와 같이 만들 수도 있습니다.

In [150]:
from langchain_core.output_parsers import JsonOutputParser

prompt1 = ChatPromptTemplate.from_template("미국 영화 배우와 대표작을 하나 나열하세요. json 형식으로 출력하고, 각 항목은 actor, movie로 표시하세요.")
prompt2 = ChatPromptTemplate.from_template(
    "{actor}는 {movie}에서 어떤 역할을 했습니까?"
)

chain1 = prompt1 | llm | JsonOutputParser()
chain2 =(
     chain1 | prompt2 | llm | StrOutputParser()
)
chain2.invoke({})

'레오나르도 디카프리오는 영화 "인셉션"에서 도미닉 "돔" 코브(Dominick "Dom" Cobb) 역할을 맡았습니다. 그는 꿈속에서 다른 사람의 아이디어를 훔치는 전문적인 도둑이지만, 이번에는 반대로 꿈속에 아이디어를 심는 \'인셉션\' 임무를 맡게 됩니다.'